### Importing modules

In [1]:
import hoomd
import hoomd.md
import gsd, gsd.hoomd
import plato.draw.vispy as draw
import ipywidgets as widgets
import random
import numpy as np
import matplotlib.pyplot as pp

### Simulation setup

In [2]:
hoomd.context.initialize("")

HOOMD-blue 2.9.2 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 06/26/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU


In [3]:
# setting parameters

# number of particles, cube root
# represents number of particles along one side of our 3D box
num_particles_cubert = 10

# lattice constant defines spacing between particles
lattice_const = 3.0

# temperature settings
startTemp = 1.0
endTemp = 0.1

# set simulation length
timeSteps = 2e6

In [4]:
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=lattice_const), n=num_particles_cubert)

# randomize velocities based on temperature
for p in system.particles:
    mass = p.mass;
    vx = random.gauss(0, startTemp / mass)
    vy = random.gauss(0, startTemp / mass)
    vz = random.gauss(0, startTemp / mass)
    p.velocity = (vx, vy, vz)

notice(2): Group "all" created containing 1000 particles


In [5]:
# Oscillating pair potential

def OPP(r, rmin, rmax, k, phi, shift, scale):
    cos = np.cos(k * (r-1) + phi)
    sin = np.sin(k * (r-1) + phi)
    V = (1/r**15 + cos/r**3 - shift)*scale
    F = (15/r**16 + 3*cos/r**4 + k*sin/r**3)*scale
    return V, F

# Determine the potential range by searching for extrema
def determineRange(k, phi):
    r = 0.5
    extrema = []
    extremaNum = 0
    force1 = OPP(r, 0, 0, k, phi, shift=0, scale=1)[1]
    while ((extremaNum < 4 or extrema[0] >= extrema[3]) and extremaNum < 6 and r < 5.0):
    #while (extremaNum < 6 and r < 5.0):
        r += 1e-5
        force2 = OPP(r, 0, 0, k, phi, shift=0, scale=1)[1]
        if (force1 * force2 < 0.0):
            extremaNum += 1
            force1 = force2
            extrema.append(OPP(r, 0, 0, k, phi, shift=0, scale=1)[0])
            print(r)
    shifted = OPP(r, 0, 0, k, phi, shift=0, scale=1)[0]
    scale = -1/(min(extrema)-shifted)
    #print([scale*(i-shifted) for i in extrema])
    return r, shifted, scale

nl = hoomd.md.nlist.cell()
table = hoomd.md.pair.table(width = 1000, nlist = nl)
r_range, shift, scale = determineRange(8.0, 4.0) #k, phi

table.pair_coeff.set('A', 'A', func = OPP, rmin = 0.5, rmax = r_range,
                coeff = dict(k=8.0, phi=4.0, shift=shift, scale=scale))

1.0446699999980171
1.2338999999992568
1.650510000001986
2.048140000004591
2.4444700000071875
2.8397900000097773


In [6]:
# set integration parameters

hoomd.md.integrate.mode_standard(dt=0.005)

all = hoomd.group.all()
hoomd.md.integrate.nvt(group=all, 
                       kT=hoomd.variant.linear_interp(points = [(0, startTemp), (timeSteps, endTemp)]), 
                       tau=1.0)

In [7]:
# record data

# for visualization
hoomd.dump.gsd("dump4.gsd", period=timeSteps*1e-2, group=all, overwrite=True)

# for plots
hoomd.analyze.log(filename="dump.log",
                  quantities=['time', 'potential_energy', 'temperature'],
                  period=timeSteps*1e-2,
                  overwrite=True)

In [8]:
# running simulation

hoomd.run(timeSteps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 1000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:11 | Step 37961 / 2000000 | TPS 3796.06 | ETA 00:08:36
Time 00:00:21 | Step 78319 / 2000000 | TPS 4035.54 | ETA 00:07:56
Time 00:00:31 | Step 119552 / 2000000 | TPS 4123.24 | ETA 00:07:36
Time 00:00:41 | Step 160679 / 2000000 | TPS 4112.68 | ETA 00:07:27
Time 00:00:51 | Step 201666 / 2000000 | TPS 4098.63 | ETA 00:07:18
Time 00:01:01 | Step 242197 / 2000000 | TPS 4053.04 | ETA 00:07:13
Time 00:01:11 | Step 283187 / 2000000 | TPS 4098.98 | ETA 00:06:58
Time 00:01:21 | Step 324209 / 2000000 | TPS 4102.15 | ETA 00:06:48
Time 00:01:31 | Step 364754 / 2000000 | TPS 4054.44 | ETA 00:06:43
Time 00:01:41 | Step 404655 / 2000000 | TPS 3990.06 | ETA 00:06:39
Time 00:01:51 | Step 443648 / 2000000 | TPS 3899.26 | ETA 00:06:39
Time 00:02:01 | Step 48

Time 00:19:51 | Step 1510405 / 2000000 | TPS 841.039 | ETA 00:09:42
Time 00:20:01 | Step 1518665 / 2000000 | TPS 825.713 | ETA 00:09:42
Time 00:20:11 | Step 1527062 / 2000000 | TPS 839.465 | ETA 00:09:23
Time 00:20:21 | Step 1535422 / 2000000 | TPS 835.941 | ETA 00:09:15
Time 00:20:31 | Step 1543851 / 2000000 | TPS 842.864 | ETA 00:09:01
Time 00:20:41 | Step 1552328 / 2000000 | TPS 847.651 | ETA 00:08:48
Time 00:20:51 | Step 1560801 / 2000000 | TPS 846.975 | ETA 00:08:38
Time 00:21:01 | Step 1569247 / 2000000 | TPS 844.493 | ETA 00:08:30
Time 00:21:11 | Step 1577745 / 2000000 | TPS 849.793 | ETA 00:08:16
Time 00:21:21 | Step 1586002 / 2000000 | TPS 825.665 | ETA 00:08:21
Time 00:21:31 | Step 1594510 / 2000000 | TPS 850.739 | ETA 00:07:56
Time 00:21:41 | Step 1602981 / 2000000 | TPS 847.027 | ETA 00:07:48
Time 00:21:51 | Step 1611472 / 2000000 | TPS 848.878 | ETA 00:07:37
Time 00:22:01 | Step 1619945 / 2000000 | TPS 847.232 | ETA 00:07:28
Time 00:22:11 | Step 1628297 / 2000000 | TPS 835

### Visualizing simulation

In [9]:
def getFrameCount(fname):
    """
    inputs: fname, the filename (ex: 'dump.gsd')
    outputs: len(traj), number of frames in simulation
    """
    with gsd.hoomd.open(fname, 'rb') as traj:
        return len(traj)

In [10]:
filename = 'dump4.gsd'
frame_num = getFrameCount(filename)

In [11]:
# function for centering droplet
def centercrystal(simulationbox,particlepositions):
    """
    Recentering the positions of all particles relative to COM, because otherwise they are split across the "box"
    Algorithm projects points along 3 orthogonal cylinders, finds COM, projects back into real space
    Useful resource w/ similar algorithm: https://www.cs.drexel.edu/~david/Papers/Bai_JGT.pdf
    
    inputs:
    simulationbox= 6x1 array of the box dimensions [Lx Ly Lz xy xz yz]
    particlepositions= the positions of the particles from the snapshot, #particles x 3 dimensions (xyz) array
    
    """    
    #center particles
    simbox=[simulationbox[3]-simulationbox[0], simulationbox[4]-simulationbox[1], simulationbox[5]-simulationbox[2]]
    theta = (particlepositions / simbox + .5) * 2 * np.pi
    sums = np.sum(np.exp(1j * theta), axis = 0)
    fractions = np.angle(sums) / 2 / np.pi
    fractions %= 1.
    fractions -= .5
    delta = fractions * simbox
    pos_CM=np.copy(particlepositions)
    pos_CM[:] -= delta[np.newaxis, :]
    
    # wrap particles back into box
    pos_CM[pos_CM[:, 0] > simulationbox[0]/2] -= [simulationbox[0], 0, 0]
    pos_CM[pos_CM[:, 1] > simulationbox[1]/2] -= [0, simulationbox[1], 0]
    pos_CM[pos_CM[:, 2] > simulationbox[2]/2] -= [0, 0, simulationbox[2]]
    pos_CM[pos_CM[:, 0] < -simulationbox[0]/2] += [simulationbox[0], 0, 0]
    pos_CM[pos_CM[:, 1] < -simulationbox[1]/2] += [0, simulationbox[1], 0]
    pos_CM[pos_CM[:, 2] < -simulationbox[2]/2] += [0, 0, simulationbox[2]]
    
    return pos_CM

In [12]:
prim = draw.Spheres()
box_prim = draw.Box(color=(0, 0, 0, 1), width=.2) #box
scene = draw.Scene([prim, box_prim], zoom=.5, clip_scale=5) #box
#scene = draw.Scene(prim) # no box
scene.show()

# looping over frames
@widgets.interact(frame_index=(0, frame_num-1, 1))
def plot(frame_index=0):
    with gsd.hoomd.open(filename, 'rb') as traj:
        frame = traj[frame_index]
        box = frame.configuration.box
        for (name, val) in zip(['Lx', 'Ly', 'Lz', 'xy', 'xz', 'yz'], box):
            setattr(box_prim, name, val)  
        
        # grabbing particle positions and diameters from simulation file
        prim.positions = centercrystal(box, frame.particles.position)
        #prim.positions = frame.particles.position
        prim.diameters = np.full(len(frame.particles.position), 1)
        
        # setting particle colors
        colors = np.ones((len(prim.positions), 4))
        colors[:, :3] = np.float32(np.divide([255, 0, 0], 255)) #pink
        prim.colors = colors
    scene.render()

VispyWidget(height=600, width=800)

interactive(children=(IntSlider(value=0, description='frame_index', max=99), Output()), _dom_classes=('widget-…

### Plotting simulation data over time

In [ ]:
data = np.genfromtxt(fname='dump.log', skip_header=True);

In [ ]:
pp.figure(figsize=(4,2.2), dpi=140)
pp.plot(data[:,1], data[:,2])
pp.xlabel('time')
pp.ylabel('potential energy')

In [ ]:
pp.figure(figsize=(4,2.2), dpi=140);
pp.plot(data[:,1], data[:,3]);
pp.xlabel('time');
pp.ylabel('temperature');